In [1]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

In [2]:
import kotlinx.coroutines.runBlocking

runBlocking {
    val channel = Channel<Int>() // 채널은 일종의 파이프
    launch {
        for (x in 1..10) {
            channel.send(x) // 송신측에서 채널에 send로 데이터를 전달
        }
    }

    repeat(10) {
        println(channel.receive()) //  수신 측에서 채널을 통해 receive 받
    }

    println("Done!!")
}

1
2
3
4
5
6
7
8
9
10
Done!!


In [3]:
// send나 receive가 suspension point이고 서로에게 의존적이기 때문에 같은 코루틴에서 사용하는 것은 위험
// 무한으로 대기에 빠짐
runBlocking {
    val channel = Channel<Int>()

    launch {
        for (x in 1..10) {
            channel.send(x)
        }

        repeat(10) {
            println(channel.receive())
        }

        println("Done!!")
    }
}


org.jetbrains.kotlinx.jupyter.exceptions.ReplInterruptedException: The execution was interrupted

In [4]:
runBlocking {
    val channel = Channel<Int>()

    launch {
        for (x in 1..10) {
            channel.send(x)
        }

        channel.close()
    }

    for (x in channel) {
        println(x)
    }

    println("Done!!")
}

1
2
3
4
5
6
7
8
9
10
Done!!


In [5]:
// 생산자(producer)와 소비자(consumer)는 굉장히 일반적인 패턴입니다. 채널을 이용해서 한 쪽에서 데이터를 만들고 다른 쪽에서 받는 것을 도와주는 확장 함수들이 있음
// - produce 코루틴을 만들고 채널을 재공
// - consumeEach 채널에서 반복해서 데이터를 제공

runBlocking {
    val oneToTen = produce {
        for (x in 1..10) {
            channel.send(x)
        }
    }

    oneToTen.consumeEach {
        println(it)
    }

    println("Done!!")
}

1
2
3
4
5
6
7
8
9
10
Done!!


In [8]:
// 파이프라인
// - 하나의 스트림을 프로듀서가 만들고, 다른 코루틴에서 그 스트림을 읽어 새로운 스트림을 만드는 패턴.
fun CoroutineScope.produceNumbers() = produce<Int> {
    var x = 1
    while (true) {
        send(x++)
    }
}

fun CoroutineScope.produceStringNumbers(numbers: ReceiveChannel<Int>): ReceiveChannel<String> = produce {
    for (i in numbers) {
        send("${i}")
    }
}

runBlocking {
    val numbers = produceNumbers()
    val stringNumbers = produceStringNumbers(numbers)

    repeat(5) {
        println(stringNumbers.receive())
    }

    println("done!!")
    coroutineContext.cancelChildren()
}

1
2
3
4
5
done!!


In [9]:
fun CoroutineScope.produceNumbers() = produce<Int> {
    var x = 1
    while (true) {
        send(x++)
    }
}

fun CoroutineScope.filterOdd(numbers: ReceiveChannel<Int>): ReceiveChannel<String> = produce {
    for (i in numbers) {
        if (i % 2 == 1) {
            send("${i}!!")
        }
    }
}

runBlocking<Unit> {
    val numbers = produceNumbers()
    val oddNumbers = filterOdd(numbers)

    repeat(10) {
        println(oddNumbers.receive())
    }

    println("done!!")
    coroutineContext.cancelChildren()
}

1!!
3!!
5!!
7!!
9!!
11!!
13!!
15!!
17!!
19!!
done!!


In [10]:
fun CoroutineScope.numbersFrom(start: Int) = produce<Int> {
    var x = start
    while(true) {
        send(x++)
    }
}

fun CoroutineScope.filter(numbers: ReceiveChannel<Int>, prime: Int): ReceiveChannel<Int> = produce{
    for (i in numbers) {
        if (i % prime != 0) {
            send(i)
        }
    }
}

runBlocking<Unit> {
    var numbers = numbersFrom(2)

    repeat(10) {
        val prime = numbers.receive()
        println(prime)
        numbers = filter(numbers, prime)
    }

    println("done!!")
    coroutineContext.cancelChildren()
}

2
3
5
7
11
13
17
19
23
29
done!!


In [11]:
fun CoroutineScope.produceNumbers() = produce<Int> {
    var x = 1
    while(true) {
        send(x++)
        delay(100L)
    }
}

fun CoroutineScope.processNumber(id: Int, channel: ReceiveChannel<Int>) = launch {
    channel.consumeEach {
        println("${id}가 ${it}을 받았습니다.")
    }
}

// 여러 코루틴이 동시에 채널을 구독
runBlocking {
    val producer = produceNumbers()

    repeat(5) {
        processNumber(it, producer)
    }

    delay(1000L)
    producer.cancel()
}

0가 1을 받았습니다.
0가 2을 받았습니다.
1가 3을 받았습니다.
2가 4을 받았습니다.
3가 5을 받았습니다.
4가 6을 받았습니다.
0가 7을 받았습니다.
1가 8을 받았습니다.
2가 9을 받았습니다.
3가 10을 받았습니다.


In [12]:
suspend fun produceNumbers(channel: SendChannel<Int>, from: Int, interval: Long) {
    var x = from
    while(true) {
        channel.send(x)
        x += 2
        delay(interval)
    }
}

fun CoroutineScope.processNumber(channel: ReceiveChannel<Int>) = launch {
    channel.consumeEach {
        println("${it}을 받았습니다.")
    }
}

// 팬 인은 반대로 생산자가 많은 것
runBlocking {
    val channel = Channel<Int>()

    launch {
        produceNumbers(channel, 1, 100L)
    }

    launch {
        produceNumbers(channel, 2, 150L)
    }

    processNumber(channel)
    delay(1_000L)
    coroutineContext.cancelChildren()
}

1을 받았습니다.
2을 받았습니다.
3을 받았습니다.
4을 받았습니다.
5을 받았습니다.
6을 받았습니다.
7을 받았습니다.
9을 받았습니다.
8을 받았습니다.
11을 받았습니다.
10을 받았습니다.
13을 받았습니다.
15을 받았습니다.
12을 받았습니다.
17을 받았습니다.
14을 받았습니다.
19을 받았습니다.


In [13]:
suspend fun someone(channel: Channel<String>, name: String) {
    for (comment in channel) {
        println("${name}: ${comment}")
        channel.send(comment.drop(1) + comment.first())
        delay(100L)
    }
}

runBlocking { // 두 개의 코루틴에서 채널을 서로 사용할 때 공정하게 기회를 줌
    val channel = Channel<String>()

    launch {
        someone(channel, "yeonuel")
    }

    launch {
        someone(channel, "seyeoun")
    }

    channel.send("sodam campus")
    delay(1_000L)
    coroutineContext.cancelChildren()
}

yeonuel: sodam campus
seyeoun: odam campuss
yeonuel: dam campusso
seyeoun: am campussod
yeonuel: m campussoda
seyeoun:  campussodam
yeonuel: campussodam 
seyeoun: ampussodam c
yeonuel: mpussodam ca
seyeoun: pussodam cam
yeonuel: ussodam camp


In [20]:
import kotlinx.coroutines.selects.*

fun CoroutineScope.saySodam() = produce<String> {
    while(true) {
        delay(100L)
        send("sodam")
    }
}

fun CoroutineScope.sayCampus() = produce<String> {
    while (true) {
        delay(150L)
        send("campus")
    }
}

// 먼저 끝나는 요청을 처리하는 것이 중요할 수 있음
// 위 경우에는 select를 사용할 수 있음
runBlocking<Unit> {
    val sodam = saySodam()
    val campus = sayCampus()
    repeat(5) {
        select<Unit>  {
            sodam.onReceive {
                println("sodam: $it")
            }

            campus.onReceive {
                println("campus: $it")
            }
        }
    }
    coroutineContext.cancelChildren()
}

sodam: sodam
campus: campus
sodam: sodam
campus: campus
sodam: sodam


In [23]:
runBlocking<Unit> {
    val channel = Channel<Int>(10)

    launch {
        for (x in 1..20) {
            println("${x} 전송중")
            channel.send(x)
        }

        channel.close()
    }

    for (x in channel) {
        println("${x} 수신")
        delay(100L)
    }

    println("Done!!")
}

1 전송중
2 전송중
3 전송중
4 전송중
5 전송중
6 전송중
7 전송중
8 전송중
9 전송중
10 전송중
11 전송중
12 전송중
1 수신
2 수신
13 전송중
3 수신
14 전송중
4 수신
15 전송중
5 수신
16 전송중
6 수신
17 전송중
7 수신
18 전송중
8 수신
19 전송중
9 수신
20 전송중
10 수신
11 수신
12 수신
13 수신
14 수신
15 수신
16 수신
17 수신
18 수신
19 수신
20 수신
Done!!


In [24]:
runBlocking {
    val channel = Channel<Int>(Channel.RENDEZVOUS) // 버퍼 사이즈를 랑데뷰(Channel.RENDEZVOUS)로 지정
    // 랑데뷰는 버퍼 사이즈를 0으로 지정하는 것

    launch {
        for (x in 1..20) {
            println("${x} 전송중")
            channel.send(x)
        }

        channel.close()
    }

    for (x in channel) {
        println("${x} 수신")
        delay(100L)
    }

    println("done!!")
}

1 전송중
2 전송중
1 수신
2 수신
3 전송중
3 수신
4 전송중
4 수신
5 전송중
5 수신
6 전송중
6 수신
7 전송중
7 수신
8 전송중
8 수신
9 전송중
9 수신
10 전송중
10 수신
11 전송중
11 수신
12 전송중
12 수신
13 전송중
13 수신
14 전송중
14 수신
15 전송중
15 수신
16 전송중
16 수신
17 전송중
17 수신
18 전송중
18 수신
19 전송중
19 수신
20 전송중
20 수신
done!!


In [26]:
/**
 * SUSPEND - 잠이 들었다 깨어남.
 * DROP_OLDEST - 예전 데이터를 지움.
 * DROP_LATEST - 새 데이터를 지움.
 */
runBlocking<Unit> {
    val channel = Channel<Int>(2, BufferOverflow.DROP_OLDEST)

    launch {
        for (x in 1..50) {
            channel.send(x)
        }

        channel.close()
    }

    delay(500L)

    for (x in channel) {
        println("${x} 수신")
        delay(100L)
    }

    println("Done!!")
}

49 수신
50 수신
Done!!
